# __Search Text by Text__

- Tutorial Difficulty: ★★☆☆☆
- 7 min read 
- Languages: [SQL](https://en.wikipedia.org/wiki/SQL) (100%)
- File location: tutorial_en/thanosql_search/search_text_by_text.ipynb   
- References: [(Kaggle) IMDB Movie Reviews](https://www.kaggle.com/code/lakshmi25npathi/sentiment-analysis-of-imdb-movie-reviews/data), [Word Embeddings: LEXICAL SEMANTICS Encoding](https://pytorch.org/tutorials/beginner/nlp/word_embeddings_tutorial.html)

## Tutorial Introduction

<div class="admonition note">
    <h4 class="admonition-title">Understanding Text Vectorization</h4>
    <p>Computers cannot directly interpret human language(natural language). Therefore, a process for converting natural language into numerical data that can be recognized by computers is required. In the field of natural language processing, embedding refers to the result of converting human natural language into a vectorized format, which is a form that can be understood by machines.</p>
</div>

Techniques for converting natural language into embeddings are largely divided into statistical techniques and artificial neural network-based techniques. ThanoSQL provides a method to train a text vectorization model using self-supervised learning.

<div class="admonition note">
    <h4 class="admonition-title">In This Tutorial</h4>
    <p>👉 Uses movie review data. The data consists of movie review texts and label values. However, because we are demonstrating self-supervised learning, this tutorial does not use label values. By training a model with 4,000 movie reviews, we will be able to search text by text and extract the importance of each word from a given movie review. </p>
</div>

## __0. Prepare Dataset__

As mentioned in the [ThanoSQL Workspace](https://docs.thanosql.ai/en/getting_started/paas/workspace/lab/), you must create an API token and run the query below to execute the query of ThanoSQL. 

In [ ]:
%load_ext thanosql
%thanosql API_TOKEN=<Issued_API_TOKEN>

### __Prepare Dataset__

In [2]:
%%thanosql
GET THANOSQL DATASET movie_review_data
OPTIONS (overwrite=True)

Success


<div class="admonition note">
    <h4 class="admonition-title">Query Details</h4>
    <ul>
        <li>"<strong>GET THANOSQL DATASET</strong>" downloads the specified dataset to the workspace.</li>
        <li>"<strong>OPTIONS</strong>" specifies the option values to be used for the <strong>GET THANOSQL DATASET</strong> clause.
        <ul>
            <li>"overwrite": determines whether to overwrite a dataset if it already exists. If set as True, the old dataset is replaced with the new dataset (bool, optional, True|False, default: False)</li>
        </ul>
        </li>
    </ul>
</div>

In [3]:
%%thanosql
COPY movie_review_train
OPTIONS (if_exists='replace') 
FROM 'thanosql-dataset/movie_review_data/movie_review_train.csv'

Success


In [4]:
%%thanosql
COPY movie_review_test 
OPTIONS (if_exists='replace') 
FROM 'thanosql-dataset/movie_review_data/movie_review_test.csv'

Success


<div class="admonition note">
    <h4 class="admonition-title">Query Details</h4>
    <ul>
        <li>"<strong>COPY</strong>" specifies the name of the dataset to be saved as a database table.</li>
        <li>"<strong>OPTIONS</strong>" specifies the option values to be used for the <strong>COPY</strong> clause.
        <ul>
           <li>"if_exists": determines how the function should handle the case where the table already exists, it can either raise an error, append to the existing table, or replace the existing table (str, optional, 'fail'|'replace'|'append', default: 'fail')</li>
        </ul>
        </li>
    </ul>
</div>

## __1. Check Dataset__

To create a movie review text search model, we use the __movie_review_train__ table located in the ThanoSQL workspace database. Run the query below to check the contents of the table.

In [5]:
%%thanosql
SELECT *
FROM movie_review_train
LIMIT 5 

,review,sentiment
0,This is the kind of movie that BEGS to be show...,negative
1,Bulletproof is quite clearly a disposable film...,negative
2,A beautiful shopgirl in London is swept off he...,positive
3,"VERY dull, obvious, tedious Exorcist rip-off f...",negative
4,Do we really need any more narcissistic garbag...,negative


<div class="admonition note">
   <h4 class="admonition-title">Understanding the Data Table</h4>
   <p>The <strong>movie_review_train</strong> table contains the following information.</p>
   <ul>
      <li>review: movie review in text format</li>
      <li>sentiment: target value indicating whether the review has a positive or negative sentiment</li>
   </ul>
</div>

## __2. Build a Text Vectorization Model__

To create a text search model with the name __movie_text_search_model__ using the __movie_review_train__ table, run the following query.  
(Estimated duration of query execution: 2 min)

In [6]:
%%thanosql
BUILD MODEL movie_text_search_model
USING SBERTEn
OPTIONS (
    text_col='review',
    overwrite=True
    )
AS
SELECT *
FROM movie_review_train

Success


<div class="admonition note">
    <h4 class="admonition-title">Query Details</h4>
    <ul>
        <li>"<strong>BUILD MODEL</strong>" creates and trains a model named <strong>movie_text_search_model</strong>.</li>
        <li>"<strong>USING</strong>" specifies <strong>SBERTEn</strong> as the base model.</li>
        <li>"<strong>OPTIONS</strong>" specifies the option values used to create a model.
        <ul>
            <li>"text_col": a column containing movie review data in the data table (str, default: 'text')</li>
            <li>"max_epochs": number of times to train with the training dataset (int, optional, default: 1)</li>
            <li>"batch_size": the size of dataset bundle utilized in a single cycle of training (int, optional, default: 16)</li> 
            <li>"overwrite": determines whether to overwrite a model if it already exists. If set as True, the old model is replaced with the new model (bool, optional, True|False, default: False) </li>
        </ul>
        </li>
    </ul>
</div>

To vectorize the __movie_review_test__ texts run the following "__CONVERT USING__" query. The vectorized results are stored in a user-defined column(default: 'convert_result') in the __movie_review_test__ table.

In [7]:
%%thanosql
CONVERT USING movie_text_search_model
OPTIONS (
    text_col='review',
    batch_size=32,
    result_col="convert_result"
    )
AS 
SELECT *
FROM movie_review_test

,review,sentiment,convert_result
0,"I read the book before seeing the movie, and t...",positive,"[b'V', b'\xdd', b'\xb5', b';', b'q', b'v', b'\..."
1,"""9/11,"" hosted by Robert DeNiro, presents foot...",positive,"[b'\x8a', b'\xc6', b'\xb2', b'<', b'\xb7', b'\..."
2,"Yesterday I attended the world premiere of ""De...",positive,"[b'\x1c', b'\x97', b'\x83', b';', b'\xee', b'\..."
3,Moonwalker is a Fantasy Music film staring Mic...,positive,"[b'\xb7', b'w', b'\x95', b'\xb9', b'\xf5', b'\..."
4,"Welcome to Oakland, where the dead come out to...",positive,"[b'\x02', b'\x8c', b'A', b'\xbb', b'\x13', b'\..."
...,...,...,...
995,(WARNING: SPOILERS!) Five young people ignore ...,positive,"[b'\x80', b'\x80', b'i', b'<', b'\x82', b'\xd9..."
996,CyberTracker is set in Los Angeles sometime in...,negative,"[b'i', b'\x9f', b'\x99', b'\xbc', b'\x84', b'C..."
997,"There is so much that is wrong with this film,...",negative,"[b'\x84', b'\x1f', b'\xa8', b'\xbc', b'k', b'\..."
998,"I am a firm believer that a film, TV serial or...",positive,"[b'\x03', b'\xea', b'0', b'=', b'v', b'd', b'\..."


<div class="admonition note">
    <h4 class="admonition-title">Query Details</h4>
    <ul>
        <li>"<strong>CONVERT USING</strong>" uses <strong>movie_text_search_model</strong> as an algorithm for text vectorizaion.</li>
        <li>"<strong>OPTIONS</strong>" specifies the options to be used for text vectorizaion.
        <ul>
            <li>"text_col": a column containing movie review data in the data table (str, default: 'text')</li>
            <li>"batch_size": the size of dataset bundle utilized in a single cycle of training (int, optional, default: 16)</li>
            <li>"result_col": defines the column name that contains the vectorized results (str, optional, default: 'convert_result')</li>
        </ul>
        </li>
    </ul>
</div>

## __3. Search for Similar Texts__

This step uses the __movie_text_search_model__ text vectorization model and test table to search for similar texts.

In [8]:
%%thanosql
SELECT review, sentiment, score
FROM (
    SEARCH TEXT 
    USING movie_text_search_model
    OPTIONS (
        search_by='text',
        search_input='This movie was my favorite movie of all time',
        emb_col='convert_result',
        result_col='score',
        top_k=10
        )
    AS 
    SELECT * 
    FROM movie_review_test
    )

,review,sentiment,score
0,This movie was the second movie I saw on the c...,positive,0.595366
1,The Muppet movie is an instant classic. I reme...,positive,0.584322
2,I have loved this movie since I saw it in the ...,positive,0.581791
3,This movie was very good because it remember w...,positive,0.580120
4,I live up here in croc territory and remember ...,positive,0.577958
5,This film lingered and lingered at a small mov...,positive,0.575489
6,So fortunate were we to see this fantastic fil...,positive,0.575422
7,We watched this in my Women's Health Issues cl...,positive,0.574574
8,What can I say?? This movie has it all...Roman...,positive,0.570684
9,"The critics were like ""a movie that will break...",negative,0.568527


In [9]:
%%thanosql
SELECT review, sentiment, score
FROM (
    SEARCH TEXT 
    USING movie_text_search_model
    OPTIONS (
        search_by='text',
        search_input='The movie was unsatisfactory',
        emb_col='convert_result',
        result_col='score',
        top_k=10
        )
    AS 
    SELECT * 
    FROM movie_review_test
    )

,review,sentiment,score
0,Just plain terrible. Nick and Michael are WAY ...,negative,0.731607
1,There was absolutely nothing in this film that...,negative,0.708185
2,"A gave it a ""2"" instead of a ""1"" (awful) becau...",negative,0.705886
3,"While the dog was cute, the film was not. It w...",negative,0.695443
4,"I knew this movie wasn't going to be amazing, ...",negative,0.692555
5,"Well, I remember when the studio sacked Schrad...",negative,0.692145
6,"If this movie would have been in English, all ...",negative,0.691321
7,4 out of 10. This film was neither funny as a ...,negative,0.685553
8,"i think this one sucked on ice, because it lef...",negative,0.681878
9,"We brought this film as a joke for a friend, a...",negative,0.680942


<div class="admonition note">
    <h4 class="admonition-title">Query Details</h4>
    <ul>
        <li>"<strong>SEARCH TEXT [image|audio|video|text|keyword]</strong>" defines the image|audio|video|text data type to search for.</li>
        <li>"<strong>USING</strong>" defines the model used for the text vectorization.</li>
        <li>"<strong>OPTIONS</strong>" specifies the options to be used for text searching.
        <ul>
            <li>"search_by": defines the image|text|audio|video type to be used for the search (str)</li>
            <li>"search_input": defines the input to be used for the search (str)</li>
            <li>"emb_col": the column that contains the vectorized results (str)</li>
            <li>"result_col": defines the name of the column that contains the search results (str, optional. default: 'search_result')</li>
            <li>"top_k": number of rows to return. If set as None, returns the entire data table (int, optional, default: 1000)</li>
        </ul>
        </li>
        <li>"<strong>AS</strong>" defines the embedding table to be used for the searches. In this example, <strong>movie_review_test</strong> table is used.</li>
    </ul>
</div>

## __4. Extract Keywords from Texts__

This step uses the __movie_text_search_model__ text vectorization model and test table to extract keywords from the texts.

In [10]:
%%thanosql
SEARCH KEYWORD
USING movie_text_search_model
OPTIONS (
    text_col='review',
    ngram_range=[1, 3],
    use_stopwords=True
    )
AS 
SELECT * 
FROM movie_review_test
LIMIT 10 OFFSET 40

,review,sentiment,convert_result,keyword
0,"This is a cute film starring Spanky, Alfalfa a...",positive,"[b'\x19', b'\xcb', b'\x88', b'<', b'&', b'0', ...","{'keyword': ['film starring spanky', 'buckwhea..."
1,"""Pecker"" is a young, unknown photographer from...",positive,"[b'p', b'V', b'\xc3', b'<', b'\r', b'\xaa', b'...","{'keyword': ['pecker great comedy', 'furlong t..."
2,Paul Lukas played a Russian intellectual makin...,positive,"[b'""', b'\x99', b'\xcb', b'<', b'w', b'\x1b', ...","{'keyword': ['actor 1930s', 'counters loretta ..."
3,"I used to LOVE this movie as a kid but, seeing...",negative,"[b'\x83', b'\x05', b'\x98', b';', b'\x11', b'\...","{'keyword': ['voice overs terrible', 'love mov..."
4,"to be honest, i didn't watch all of the origin...",negative,"[b'\x0b', b'S', b'\r', b'<', b'^', b'A', b'\xf...","{'keyword': ['vampire movie', 'didn watch orig..."
5,Police Squad! (1982) was a funny show that end...,positive,"[b'\x96', b'w', b'\xc7', b'\xba', b'\x02', b' ...","{'keyword': ['comedian short lived', 'american..."
6,I am still shuddering at the thought of EVER s...,negative,"[b'\x1d', b'\x13', b'\x9e', b'\xbc', b'`', b'\...","{'keyword': ['movie stupid', 'point movie fact..."
7,Gregory Peck gives a brilliant performance in ...,positive,"[b'8', b'\x82', b'\xdf', b'<', b'{', b'\xd5', ...","{'keyword': ['brilliant performance film', 'aw..."
8,I first flicked onto the LoG accidentally one ...,positive,"[b'\xdc', b'\x17', b'F', b'\xbb', b'\n', b'\xa...","{'keyword': ['episode humour weird', 'commenta..."
9,A family looking for some old roadside attract...,negative,"[b'\xea', b'\xcd', b'\xbb', b'<', b'\x02', b'\...","{'keyword': ['forgettable poorly horror', 'hil..."


In [11]:
%%thanosql
SELECT review, sentiment, keyword -> 'keyword' AS keywords, keyword -> 'score' AS score
FROM (
    SEARCH KEYWORD 
    USING movie_text_search_model
    OPTIONS (
        text_col='review',
        use_stopwords=True
        )
    AS 
    SELECT * 
    FROM movie_review_test
    LIMIT 10
    )

,review,sentiment,keywords,score
0,"I read the book before seeing the movie, and t...",positive,"[best adaptations, film haunting, sarandon ama...","[0.4069, 0.3494, 0.3476, 0.3334, 0.214]"
1,"""9/11,"" hosted by Robert DeNiro, presents foot...",positive,"[2001, deniro presents, 11 hosted, television ...","[0.4127, 0.3907, 0.3777, 0.3733, 0.2759]"
2,"Yesterday I attended the world premiere of ""De...",positive,"[rape scene, innocent flirtation, quest woman,...","[0.4687, 0.2807, 0.2692, 0.2376, 0.2368]"
3,Moonwalker is a Fantasy Music film staring Mic...,positive,"[film liked, jackson music, clip michael, rati...","[0.4725, 0.3591, 0.2722, 0.271, 0.2421]"
4,"Welcome to Oakland, where the dead come out to...",positive,"[carpenter haunting, zombie ricky, bourgeois m...","[0.4413, 0.4073, 0.3459, 0.3428, 0.3249]"
5,Tipping the Velvet (2002) (TV) was directed by...,positive,"[wonderful acting, protagonist nan, played rac...","[0.4911, 0.411, 0.3788, 0.3225, 0.2997]"
6,The Stock Market Crash of 1929 and the Depress...,positive,"[musicals career, favorite busby, cagney movie...","[0.4178, 0.3474, 0.3276, 0.3152, 0.2569]"
7,I want to clarify a few things. I am not famil...,negative,"[film, believe provocative, girl desperation, ...","[0.5089, 0.3581, 0.3338, 0.2645, 0.236]"
8,This is a nice movie with good performances by...,positive,"[good performances, spanish cinema, movie bori...","[0.5143, 0.514, 0.3938, 0.3241, 0.2006]"
9,"Once a month, I invite a few friends over for ...",negative,"[retarded movie, zombie personally, personally...","[0.5385, 0.3905, 0.2796, 0.2625, 0.2346]"


In [12]:
%%thanosql
SELECT review, sentiment, json_array_elements(keyword -> 'keyword') AS keywords, json_array_elements(keyword -> 'score') AS score
FROM (
    SEARCH KEYWORD 
    USING movie_text_search_model
    OPTIONS (
        text_col='review',
        use_stopwords=True,
        threshold=0.5
        )
    AS 
    SELECT * 
    FROM movie_review_test
    LIMIT 10
    )

,review,sentiment,keywords,score
0,I want to clarify a few things. I am not famil...,negative,film,0.5089
1,This is a nice movie with good performances by...,positive,good performances,0.5143
2,This is a nice movie with good performances by...,positive,spanish cinema,0.5140
3,"Once a month, I invite a few friends over for ...",negative,retarded movie,0.5385


<div class="admonition note">
    <h4 class="admonition-title">Query Details</h4>
    <ul>
        <li>"<strong>SEARCH KEYWORD</strong>" uses the <strong>movie_text_search_model</strong> as an algorithm to search keywords.</li>
        <li>"<strong>USING</strong>" defines the model used for the text vectorization.</li>
        <li>"<strong>OPTIONS</strong>" specifies the options to be used for the text vectorizaion.
            <ul>
                <li>"lang": language to use (str, optional, 'ko'|'en' default: 'ko')</li>
                <li>"text_col": a column containing movie review data in the data table (str, default: 'text')</li>
                <li>"ngram_range": minimum and maximum number of words for each keyword ex) [1, 3]. In most situations, keywords are extracted according to the maximum number of words (list[int, int], optional, default: [1, 2])</li>
                <li>"top_n": number of keywords to be extracted, in order of highest similarity (int, optional, default: 5)</li>
                <li>"diversity": variety of keywords to be extracted. The higher the value, the more diverse the keywords will be 0 <= diversity <= 1 (float, optional, default: 0.5)</li>
                <li>"use_stopwords": whether to exclude words that do not have a significant meaning (bool, optional, True|False, default: True)</li>
                <li>"threshold": minimum value of similarity value of keywords to be extracted (float, optional, default: 0.0)</li>
            </ul>
            </li>
        <li>"<strong>AS</strong>" defines the embedding table to be used for searches. In this example, <strong>movie_review_test</strong> table is used.</li>
    </ul>
</div>

## __5. Combine the Two Methods__

In [13]:
%%thanosql
SEARCH KEYWORD
USING movie_text_search_model
OPTIONS (
    text_col='review',
    ngram_range=[1, 3],
    use_stopwords=True
    )
AS (
    SELECT review, sentiment, score
    FROM (
        SEARCH TEXT 
        USING movie_text_search_model
        OPTIONS (
            search_by='text',
            search_input='The greatest movie of all time',
            emb_col='convert_result',
            result_col='score',
            top_k=10
            )
        AS 
        SELECT * 
        FROM movie_review_test
        )
    )

,review,sentiment,score,keyword
0,This is the kind of film for a snowy Sunday af...,positive,0.575603,"{'keyword': ['hours wonderful performances', '..."
1,This is without a doubt the most poorly though...,negative,0.571998,"{'keyword': ['poorly thought movie', 'history ..."
2,"This film immediately catches the eye, with th...",positive,0.565412,"{'keyword': ['moving compelling story', 'film ..."
3,The beginning of this film is a little clunky ...,positive,0.553026,"{'keyword': ['story fine performances', 'lumet..."
4,A labor of love. Each frame is picture perfect...,positive,0.551542,"{'keyword': ['recommend film appreciates', 'lo..."
5,"This movie is full of references. Like ""Mad M...",positive,0.551175,"{'keyword': ['lorre movie masterpiece', 'movie..."
6,For those out there that like to think of them...,positive,0.548396,"{'keyword': ['modest paced movie', 'film poor'..."
7,There are not many movies around that have giv...,positive,0.542594,"{'keyword': ['wonderful fantasy spectacle', 's..."
8,This is without a doubt the greatest film ever...,positive,0.541673,"{'keyword': ['recursive filmmaking triumph', '..."
9,The movie Angels of the Universe is a pure mas...,positive,0.540057,"{'keyword': ['best foreign film', 'sigurrós sy..."


In [14]:
%%thanosql
SELECT review, sentiment, keyword -> 'keyword' AS keywords, keyword -> 'score' AS score
FROM ( 
    SEARCH KEYWORD
    USING movie_text_search_model
    OPTIONS (
        text_col='review',
        ngram_range=[1, 3],
        use_stopwords=True
        )
    AS (
        SELECT review, sentiment, score
        FROM (
            SEARCH TEXT 
            USING movie_text_search_model
            OPTIONS (
                search_by='text',
                search_input='The greatest movie of all time',
                emb_col='convert_result',
                result_col='score',
                top_k=10
                )
            AS 
            SELECT * 
            FROM movie_review_test
            )
        )
    )

,review,sentiment,keywords,score
0,This is the kind of film for a snowy Sunday af...,positive,"[hours wonderful performances, film snowy sund...","[0.6337, 0.5651, 0.484, 0.3366, 0.3178]"
1,This is without a doubt the most poorly though...,negative,"[poorly thought movie, history invention gags,...","[0.6018, 0.4019, 0.3485, 0.256, 0.2114]"
2,"This film immediately catches the eye, with th...",positive,"[moving compelling story, film havoc violence,...","[0.5937, 0.5084, 0.447, 0.4279, 0.285]"
3,The beginning of this film is a little clunky ...,positive,"[story fine performances, lumet outstanding fi...","[0.6526, 0.6095, 0.553, 0.4633, 0.417]"
4,A labor of love. Each frame is picture perfect...,positive,"[recommend film appreciates, long movie, story...","[0.5799, 0.4949, 0.409, 0.3816, 0.221]"
5,"This movie is full of references. Like ""Mad M...",positive,"[lorre movie masterpiece, movie references lik...","[0.6052, 0.5778, 0.2644, 0.2565, 0.2372]"
6,For those out there that like to think of them...,positive,"[modest paced movie, film poor, enjoy indie mo...","[0.5983, 0.5148, 0.4938, 0.3543, 0.2899]"
7,There are not many movies around that have giv...,positive,"[wonderful fantasy spectacle, stardust definit...","[0.5395, 0.5051, 0.5014, 0.3937, 0.1748]"
8,This is without a doubt the greatest film ever...,positive,"[recursive filmmaking triumph, film nearly inc...","[0.5787, 0.5561, 0.4423, 0.3345, 0.2554]"
9,The movie Angels of the Universe is a pure mas...,positive,"[best foreign film, sigurrós symbolic film, di...","[0.6096, 0.5272, 0.391, 0.3905, 0.3834]"


<div class="admonition note">
    <h4 class="admonition-title">Query Details</h4>
    <ul>
        <li>"<strong>SEARCH TEXT [image|audio|video|text|keyword]</strong>" defines the image|audio|video|text data type to search for.</li>
        <li>"<strong>SEARCH KEYWORD</strong>" uses the <strong>movie_text_search_model</strong> as an algorithm to search keywords.</li>
        <li>"<strong>USING</strong>" defines the model used for text vectorization.</li>
        <li>"<strong>AS</strong>" defines the embedding table to be used for searches. In this example, <strong>movie_review_test</strong> table is used.</li>
    </ul>
</div>

## __6. In Conclusion__

In this tutorial, we performed text vectorization using movie review data, and similar text search and keyword extraction. As this is a beginner-level tutorial, we focused on the process rather than accuracy. The model's accuracy can be improved by adjusting various options, such as increasing the epoch or dataset size. Create your own model and provide competitive services by combining various unstructured data (image, audio, video, etc.) and structured data with ThanoSQL.

* [How to Upload My Data to the ThanoSQL Workspace](https://docs.thanosql.ai/en/getting_started/data_upload/)
* [How to Create a Table Using My Data](https://docs.thanosql.ai/en/how-to_guides/ThanoSQL_query/COPY_SYNTAX/)
* [How to Upload My Model to the ThanoSQL Workspace](https://docs.thanosql.ai/en/how-to_guides/ThanoSQL_query/UPLOAD_MODEL_SYNTAX/)

<div class="admonition tip">
    <h4 class="admonition-title">Inquiries About Deploying a Model for Your Own Service</h4>
    <p>If you have any difficulties creating your own model using ThanoSQL or applying it to your services, please feel free to contact us below😊</p>
    <p>For inquiries regarding building an text similarity search models: <a href="mailto:contact@smartmind.team">contact@smartmind.team</a></p>
</div>